# Sklearn Pipelines Exercise
*Made by viga@itu.dk and thso@itu.dk*

## Introduction

In this exercise you'll be working with the [Wine Quality Dataset](https://archive.ics.uci.edu/ml/datasets/wine+quality) from the UCI Machine Learning Repository. The dataset consists of 11 features and a quality score for 4898 white wine samples and 1599 red wine samples. The goal is to predict the quality of the wine based on the features.

The datasets are located in the `data` folder. The `winequality-red.csv` file contains the red wine samples and the `winequality-white.csv` file contains the white wine samples. Lastly, the `winequality.names` file contains a description of the dataset.

The goal of this exercise is to get you familiar with the [Scikit-learn Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) API. You'll be using pipelines to perform feature scaling and feature selection.

## Load in the data

You can either load the red-wine dataset or the white-wine dataset. You can also load both datasets and combine them if you want.

Both datasets are available in the `data` folder, and are called `winequality-red.csv` and `winequality-white.csv`.

Hint: You can use the `pd.read_csv()` function to load in the data (remember to check the delimiter!). You can find the documentation [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html).


In [9]:
import pandas as pd
# load in the data
data=pd.read_csv("data/winequality-white.csv",sep=(";"))

# if you want to you can combine the two datasets into one - but this is not necessary

# check a few rows of the data - hint: use .head()


## Data Exploration

### Check the number of missing values in the dataset.

Hint: `.isnull()`

Dont worry if there are missing values, we'll handle them later in our pipeline!

In [11]:
data.isnull()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,False,False,False,False,False,False,False,False,False,False,False,False
4894,False,False,False,False,False,False,False,False,False,False,False,False
4895,False,False,False,False,False,False,False,False,False,False,False,False
4896,False,False,False,False,False,False,False,False,False,False,False,False


### Check some basic statistics

We want to know the mean, standard deviation, minimum, maximum and quartiles of each feature.
This will give us a good idea of the distribution of the data, and also tell us if we need to do any scaling.

Hint: `.describe()`, If the output is hard to read, you can use `.T` to transpose the dataframe, i.e., swapping the rows and columns.

Do you notice anything strange about the data? Is there anything that stands out to you?

In [13]:
# check some basic statistics
data.describe()


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,4895.000000,4895.000000,4893.000000,4892.000000,4892.000000,4895.000000,4894.000000,4897.000000,4895.000000,4895.000000,4895.000000,4898.000000
mean,6.855148,0.278252,0.334224,6.391977,0.045781,35.306844,138.394463,0.994027,3.188319,0.489841,10.514419,5.877909
std,0.843928,0.100782,0.120905,5.073363,0.021859,17.004703,42.490153,0.002991,0.151024,0.114137,1.230389,0.885639
min,3.800000,0.080000,0.000000,0.600000,0.009000,2.000000,9.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.300000,0.210000,0.270000,1.700000,0.036000,23.000000,108.000000,0.991720,3.090000,0.410000,9.500000,5.000000
50%,6.800000,0.260000,0.320000,5.200000,0.043000,34.000000,134.000000,0.993740,3.180000,0.470000,10.400000,6.000000
75%,7.300000,0.320000,0.390000,9.900000,0.050000,46.000000,167.000000,0.996100,3.280000,0.550000,11.400000,6.000000
max,14.200000,1.100000,1.660000,65.800000,0.346000,289.000000,440.000000,1.038980,3.820000,1.080000,14.200000,9.000000


We saw that there were some missing values in the dataset, this we can fix in the pipeline, using the [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) from sklearn.

Next we also saw that there was a some differences in the scale of the different variables, so we will use the [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) from sklearn to scale the data. This will make it easier for the model to learn the patterns in the data. Especially for the KNN algorithm (which we'll use), which is based on distance, it is important that the data is scaled.

If you think of other transformations that might be useful for this dataset, feel free to try them out!

**Take a look at the [sklearn.preprocessing](https://scikit-learn.org/stable/modules/preprocessing.html) module for some inspiration.**

In [21]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

import numpy as np

# Create a SimpleImputer instance
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

# Fit and transform the data
imputed_data = imputer.fit_transform(data)

#create scaler
scaler = StandardScaler(copy=True, with_mean=True, with_std=True)


scaled_data=scaler.fit_transform(imputed_data)

Df=pd.DataFrame(scaled_data)
Df.describe()





,0,1,2,3,4,5,6,7,8,9,10,11
count,4.898000e+03,4.898000e+03,4.898000e+03,4.898000e+03,4.898000e+03,4.898000e+03,4.898000e+03,4.898000e+03,4.898000e+03,4.898000e+03,4.898000e+03,4.898000e+03
mean,-1.856870e-16,-3.655712e-16,1.044489e-16,-6.310455e-17,-2.321087e-17,2.284820e-16,-1.972924e-16,2.049520e-14,2.506774e-15,1.537720e-16,-5.338500e-16,2.901359e-17
std,1.000102e+00,1.000102e+00,1.000102e+00,1.000102e+00,1.000102e+00,1.000102e+00,1.000102e+00,1.000102e+00,1.000102e+00,1.000102e+00,1.000102e+00,1.000102e+00
min,-3.621633e+00,-1.967939e+00,-2.766061e+00,-1.142461e+00,-1.683879e+00,-1.959485e+00,-3.046837e+00,-2.313072e+00,-3.102216e+00,-2.365154e+00,-2.044431e+00,-3.249862e+00
25%,-6.580835e-01,-6.775020e-01,-5.315250e-01,-9.254874e-01,-4.477954e-01,-7.240275e-01,-7.156950e-01,-7.707374e-01,-6.512782e-01,-6.998035e-01,-8.248068e-01,-9.913739e-01
50%,-6.537366e-02,-1.811803e-01,-1.177220e-01,-2.351161e-01,-1.273292e-01,-7.688330e-02,-1.034759e-01,-9.445068e-02,-5.510431e-02,-1.739034e-01,-9.303199e-02,1.378701e-01
75%,5.273362e-01,4.144058e-01,4.616022e-01,6.919538e-01,1.931369e-01,6.290922e-01,6.735713e-01,6.930179e-01,6.073112e-01,5.272968e-01,7.200511e-01,1.378701e-01
max,8.706732e+00,8.157025e+00,1.097220e+01,1.171817e+01,1.374428e+01,1.492510e+01,7.101871e+00,1.503130e+01,4.184355e+00,5.172748e+00,2.996684e+00,3.525602e+00


# Splitting the data

Now that we have created our pipeline, we can train the model.

First we need to split the data into a training set and a test set. We will use the [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function from sklearn to do this. But first we need to split the data into features and labels.

The features are all the columns in the dataset, except for the `quality` column, which are the labels.

We will use the default split of 75% training data, and 25% test data.

Hint: You can use the `random_state` parameter to make sure that the data is split the same way every time you run the code.

The train_test_split function returns four values, the first two are the training and test data, and the last two are the train and test labels.

In [33]:
from sklearn.model_selection import train_test_split
# split the data into X and y

X_train, X_test, Y_train, Y_test = train_test_split(Df.drop(11, axis=1), Df[11], test_size=0.25, random_state=42)


# now split the data into train and test data


# Creating the pipeline

We will now create a pipeline that will handle the missing values and scaling for us, and finally train a KNN model on the data.

We will use the [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) class from sklearn to create our pipeline.

The pipeline will consist of three steps, the first step will be to impute the missing values, and the second step will be to scale the data, and the third step will be to train the model.

The pipeline format is a list of tuples, where the first element in the tuple is the name of the step, and the second element is the step itself, e.g.:

```python
pipeline = Pipeline([
	('step_name', step()),
	('step_name', step()),
	('step_name', step()),
])
```

Where the `step_name` is a string, and the `step` is a sklearn object - this can be a "Transformer" object (like `SimpleImputer` and `StandardScaler`) or an "Estimator" object (like `KNeighborsClassifier` or `LinearRegression`).

In [43]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier

# create your pipeline
pipe = Pipeline([
    ('impute', SimpleImputer()),
	('scale', StandardScaler()),
	('classify', KNeighborsClassifier()),
])

fresh_data=pd.read_csv("data/winequality-red.csv",sep=(";"))
X_train, X_test, Y_train, Y_test = train_test_split(fresh_data.drop("quality", axis=1), fresh_data["quality"], test_size=0.25, random_state=42)


pipe.fit(X_train, Y_train).score(X_test, Y_test)

0.59

# Evaluating the model

Now that we have trained the model, we want to evaluate it to see how well it performs.

We will use the [accuracy_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) function from sklearn to calculate the accuracy of the model.

Since we have created a pipeline, we can simply call the `.fit()` and `.predict()` methods on the pipeline object, and it will handle the preprocessing for us - and importantly in the correct order.

Remember to only call `.fit()` on the training data. Calling `.fit()` on the test data will cause the model to overfit to the test data, and will give you an overly optimistic accuracy score.

* **`.fit(X_train, y_train)` will train the model on the training data.**
* **`.predict(X_train)` will return the predicted labels for the test data, which you can then pass to the `accuracy_score` function, along with the true labels (y_train).**
* **`.predict(X_test)` will return the predicted labels for the test data, which you can then pass to the `accuracy_score` function, along with the true labels (y_test).**

In [44]:
from sklearn.metrics import accuracy_score
# fit the pipeline

pipe.fit(X_train,Y_train)
result1=pipe.predict(X_train)
print(accuracy_score(result1,Y_train))
# evaluate the pipeline
result2=pipe.predict(X_test)
print(accuracy_score(result2,Y_test))




0.7114261884904087
0.59
